# AD-PYNE: How-to Guide
## Automatic Differentiation Package for Python

## Introduction

**AD-PYNE** allows you to calculate the derivatives of complex functions by first instantiating simple `AutoDiff` objects and then building the desired functions using elementary functions and the instantiated `AutoDiff` objects.

## Table of Contents
1. [Importing](#importing)
2. [Forward Mode](#forward_mode)
    1. [Scalar Functions with Single Variable of Scalar](#scalar_single)
    2. [Scalar Functions with Single Variable of Vectors](#scalar_vector)
    3. [Scalar Functions with Multiple Variables of Scalars](#scalar_multiple)
    4. [Vector Functions with Single Variable of Scalar](#vector_single)
    5. [Vector Functions with Multiple Variables of Scalars](#vector_multiple)
    6. [Vector Functions with Multiple Variables of Vectors](#vector_multiple_vector)
3. [Dual Mode for Higher Order Derivatives](#dual_mode)
    1. [Higher Order Derivatives for Single Variables](#dual_high_single)
    2. [Higher Order Derivatives for Single Variables of Vectors](#dual_high_vector)
    3. [Higher Order Derivatives for Vector Functions of Single Variables](#dual_high_vector_function)

<a id='importing'></a>
## Importing

In [1]:
import numpy as np
from ADPYNE.AutoDiff import AutoDiff, vectorize
import ADPYNE.elemFunctions as ef
from ADPYNE.Dual import Dual, vectorizeDual

<a id='forward_mode'></a>
## Calculating Derivatives using Forward Mode

<a id='scalar_single'></a> 
### Derivatives of Scalar Functions of Scalars with Single Input

#### Instantiating an AutoDiff Object
If your goal is to build and find the derivatives of scalar functions of a single input, follow the template below for instantiating an `AutoDiff` object. 

In [2]:
x = AutoDiff(5, 2)

The first argument in the `AutoDiff` initialization function is the value of the function. The second argument in the `AutoDiff` initialization function is the derivative of the function. When first instantiating an AutoDiff object, this derivative is the seed. (See the **Calculating the Jacobian** section below for more detail on picking a seed.)

Each `AutoDiff` object holds the value in `.val`, the derivative in `.der`, and the Jacobian in `.jacobian`. You can access these elements as shown below.

<div class="alert alert-block alert-info">
<b>Warning:</b> All values are stored as numpy arrays. Scalar values are stored as 1 x 1 arrays. If you need to work with Python ints or floats, you must do the conversion yourself on the returned values. Do not convert the stored elements in the objects themselves.
</div>

In [3]:
print("Value: ", x.val)
print("Derivative: ", x.der)
print("Jacobian: ", x.jacobian)

Value:  [[5]]
Derivative:  [[2.]]
Jacobian:  [[1.]]


You can converting the returned elements to Python ints or floats for your own purposes.

In [4]:
# Correct conversion
theValueF = float(x.val)
theDerivativeF = float(x.der)
theJacobianF = float(x.jacobian)

In [5]:
print("Value: ", theValueF)
print("Derivative: ", theDerivativeF)
print("Jacobian: ", theJacobianF)

Value:  5.0
Derivative:  2.0
Jacobian:  1.0


<div class="alert alert-block alert-danger">
<b>Example of incorrect conversion:</b>
<br>x.val = float(x.val)
<br>x.der = float(x.der)
<br>x.jacobian = float(x.jacobian)
</div>


#### Calculating the Jacobian

You have two options for calculating the jacobian of a function. 

##### Option 1 - Using a Seed of 1
If you know that you only need the jacobian of function, you can simply initialize an `AutoDiff` object with a derivative (or seed) of 1. In this case, the derivative and jacobian of the function will be the same. 

In [6]:
x = AutoDiff(5, 1)

print("Value: ", x.val)
print("Derivative: ", x.der)
print("Jacobian: ", x.jacobian)

Value:  [[5]]
Derivative:  [[1.]]
Jacobian:  [[1.]]


##### Option 2 - Using any Seed

If you need to set the derivative of the initial `AutoDiff` object to some number other than 1, you can still find the jacobian at any time by accessing the element `.jacobian` of the `AutoDiff` object as shown in **Instantiating an AutoDiff Object**. 

#### Building Up a Function
You may build up a function with any combination of the following elementary functions. 

Some of these functions are available as part of the **AutoDiff** module and do not require importing the additional **elmeFunctions** module. This functions are as follows: addition, subtraction, multiplication, division, power, absolute value, negation, and invert.

All other elmentary functions require importing the **elemFunctions** module. These functions are: (natural) exponential, (natural) log, log base 10, square root, absolute value, sine, cosine, tangent, arc sine, arc cosine, arc tangent, hyperbolic sine, hyperbolic cosine, hyperbolic tangent, hyperbolic arc sine, hyperbolic arc cosine, and hyperbolic arc tangent.

To build up the function, pass in the AutoDiff object into the elemenary functions or manipulate the AutoDiff object using Python operations. Store the function in a variable. 

##### Example 1

In [7]:
x = AutoDiff(3, 2)
f = x**3 - 5*x**-2 + 2*x + 5

In [8]:
print("Value: ", f.val)
print("Derivative: ", f.der)
print("Jacobian: ", f.jacobian)

Value:  [[37.44444444]]
Derivative:  [[58.74074074]]
Jacobian:  [[29.37037037]]


In the example above, the AutoDiff object `x` is initialized with a value of 3 and a seed of 2 for its derivative. Each operation on `x` produces a new AutoDiff function. Any number of elementary functions and operations can be done in a single line and stored in a single AutoDiff function. 

##### Example 2

Building up the function piece by piece is also valid and will produce the same results. 

In [9]:
x = AutoDiff(3, 2)
f = x**3
g = -5*x**-2
h = 2*x + 5
fgh = f + g + h

In [10]:
print("Value: ", fgh.val)
print("Derivative: ", fgh.der)
print("Jacobian: ", fgh.jacobian)

Value:  [[37.44444444]]
Derivative:  [[58.74074074]]
Jacobian:  [[29.37037037]]


##### Example 3
Call and use the elmentary functions in the **elemFunctions** as you would any other function.

In [12]:
x = AutoDiff(np.pi, 0.5, 1)

f = ef.sin(x) + ef.cos(x**2) + ef.tan(x)*ef.sqrt(x)

print("Value: ", f.val)
print("Derivative: ", f.der)
print("Jacobian: ", f.jacobian)

Value:  [[-0.90268536]]
Derivative:  [[1.73805807]]
Jacobian:  [[3.47611614]]


<a id='scalar_vector'></a> 
### Derivatives of Scalar Functions of Vector Inputs for Single Variable

This section assumes you have some familiarity with the concepts covered in [Scalar Functions with Single Input](#scalar_single).

#### Instantiating an AutoDiff Object

You may instantiate an AutoDiff object for a scalar function with a vector of inputs for a single variable. 

For example, you would like to evaluate the derivative of a function at the following points: x = \[-2, -1, 0, 1, 2\]. 

In [12]:
x = AutoDiff([-2, -1, 0, 1, 2], 2)

In [13]:
print("Value:\n", x.val)
print("Derivative:\n", x.der)
print("Jacobian:\n", x.jacobian)

Value:
 [[-2]
 [-1]
 [ 0]
 [ 1]
 [ 2]]
Derivative:
 [[2.]
 [2.]
 [2.]
 [2.]
 [2.]]
Jacobian:
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


#### Building Up a Function
You can build up a scalar function with vector inputs as you would a scalar function with a scalar input. 

In [14]:
f = x**3 - 5*x**-2 + 2*x + 5
print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[-8.25]
 [-3.  ]
 [ -inf]
 [ 3.  ]
 [15.75]]
Derivative:
 [[ 25.5]
 [-10. ]
 [  inf]
 [ 30. ]
 [ 30.5]]
Jacobian:
 [[12.75]
 [-5.  ]
 [  inf]
 [15.  ]
 [15.25]]


A warning will appear if any of the derivatives cannot be evaluating because of dividing by 0. 

<a id='scalar_multiple'></a> 
### Derivatives of Scalar Functions of Multiple Variables of Scalars

This section assumes you have some familiarity with the concepts covered in [Scalar Functions with Single Input](#scalar_single).

#### Instantiating an AutoDiff Object

You may instantiate an AutoDiff object for a scalar function with multiple variables of scalars. 

For example, you would like to evaluate the partial derivatives of the following function:
$ f(x, y, z) = 3x^2 + xy^2 - 2z^3 $

You may instantiate an AutoDiff object for each variable $x$, $y$, $z$. You must instantiate a value and seed for each variable. You must pass in the total number of variables you will ultimately use in your function(s). You must also denote that the variable is the $k^{th}$ variable in the function.

In [19]:
x = AutoDiff(3, 2, n=3, k=1)

In this way, the AutoDiff object knows that the value and derivative of $x$ should be the first column in the jacobian. 

In [24]:
y = AutoDiff(-1, 2, n=3, k=2)
z = AutoDiff(1, 2, n=3, k=3)

The value and deriative of $y$ is the second column in the jacobian. The value and derivative of $z$ is the third column in the jacobian.

#### Building up a Function

In [26]:
f = 3*x**2 + x*(y**2) - 2*z**3
print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[28.]]
Derivative:
 [[ 38. -12. -12.]]
Jacobian:
 [[19. -6. -6.]]


The partial derivatives of $x$,$y$, and $z$ are the first, second, and third columns, respectively. 


$\left[\begin{array}{ccc}
\frac{\partial f}{\partial x} && \frac{\partial f}{\partial y} 
&& \frac{\partial f}{\partial z}
\end{array}\right]$

<a id='vector_single'></a> 
### Derivatives of Vector Functions of Single Variables with Scalar Inputs

This section assumes you have some familiarity with the concepts covered in [Scalar Functions with Single Input](#scalar_single).

#### Instantiating an AutoDiff Object
You may instantiate an AutoDiff object for a vector function with a single input. 

For example, you would like to evaluate the derivative the following vector function:

$\left[\begin{array}{c}
x^2 + 3x - 2\\
e^{-10x} \\
cos(x) + sin(x)
\end{array}\right]$

You would instantiate the AutoDiff object as before.

In [15]:
x = AutoDiff(0, 2)

#### Building Up a Vector Function

You may build up a vector function in a variety fo ways. 

##### Example 1
You may build up a vector function by building each function individually and then calling the function `vectorize`. 

In [16]:
f1 = x**2 + 3*x - 2
f2 = ef.exp(-10*x)
f3 = ef.cos(x) + ef.sin(x)

f = vectorize([f1, f2, f3])

print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[-2.]
 [ 1.]
 [ 1.]]
Derivative:
 [[  6.]
 [-20.]
 [  2.]]
Jacobian:
 [[  3.]
 [-10.]
 [  1.]]


The derivative is given in the following form:

$\left[\begin{array}{c}
\frac{\partial f}{\partial x}  \\
\frac{\partial f}{\partial x} \\
\frac{\partial f}{\partial x}
\end{array}\right]$

##### Example 2
You may build up a vector function by creating a list of functions and calling the function `vectorize` on that list.

In [17]:
fs = [x**2 + 3*x - 2, ef.exp(-10*x), ef.cos(x) + ef.sin(x)]

f = vectorize(fs)

print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[-2.]
 [ 1.]
 [ 1.]]
Derivative:
 [[  6.]
 [-20.]
 [  2.]]
Jacobian:
 [[  3.]
 [-10.]
 [  1.]]


<a id='vector_multiple'></a> 
### Derivatives of Vector Functions of Multiple Variables with Scalar Inputs

This section assumes you have some familiarity with the concepts covered in [Scalar Functions with Single Variable of Scalar](#scalar_single) and [Vector Functions with Single Variable of Scalar](#vector_single)

#### Instantiating an AutoDiff Object
You may instantiate an AutoDiff object for a vector function with a multiple scalar inputs. 

For example, you would like to evaluate the partial derivatives for the following vector function:

$\left[\begin{array}{c}
3x^2 + xy^2 - 2z^3\\
ye^{x} \\
cos(xz) + sin(\frac{y}{z})
\end{array}\right]$

You would instantiate the AutoDiff object as before in the multiple variable case.

In [2]:
x = AutoDiff(3, 2, n=3, k=1)
y = AutoDiff(-2, 2, n=3, k=2)
z = AutoDiff(1, 2, n=3, k=3)

#### Building Up a Vector Function
Create functions as before.

In [4]:
f1 = 3*x**2 + x*y**2 - 2*z**3
f2 = y*ef.exp(x)
f3 = ef.cos(x*z) + ef.sin(y/z)

When using the vectorize function, pass in the number of inputs.

In [5]:
f = vectorize([f1, f2, f3], n_inputs = 3)

print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[ 37.        ]
 [-40.17107385]
 [ -1.89928992]]
Derivative:
 [[ 44.         -24.         -12.        ]
 [-80.34214769  40.17107385   0.        ]
 [ -0.28224002  -0.83229367  -2.51130739]]
Jacobian:
 [[ 22.         -12.          -6.        ]
 [-40.17107385  20.08553692   0.        ]
 [ -0.14112001  -0.41614684  -1.2556537 ]]


The derivative is given in the following form:

$\left[\begin{array}{ccc}
\frac{\partial f_1}{\delta x} && \frac{\partial f_1}{\partial y} 
&& \frac{\partial f_1}{\partial z} \\
\frac{\partial f_2}{d x} && \frac{\partial f_2}{\partial y} 
&& \frac{\partial f_2}{d z} \\
\frac{\partial f_3}{\partial x} && \frac{\partial f_3}{\partial y} 
&& \frac{\partial f_3}{\partial z}
\end{array}\right]$

<a id='vector_multiple_vector'></a> 
### Derivatives of Vector Functions of Multiple Variables with Vector Inputs

This section assumes you have some familiarity with the concepts covered in [Scalar Functions with Single Variable of Scalar](#scalar_single), [Vector Functions with Single Variable of Scalar](#vector_single), and [Scalar Functions with Multiple Variables of Scalars](#scalar_multiple).

#### Instantiating an AutoDiff Object
You may instantiate an AutoDiff object for a vector function with a multiple vector inputs. (At the moment, Forward Mode only supports multiple inputs with vectors of the same length.)

For example, you would like to evaluate the partial derivatives for the following vector function:

$\left[\begin{array}{c}
3x^2 + xy^2 - 2z^3\\
ye^{x} \\
cos(xz) + sin(\frac{y}{z})
\end{array}\right]$

You would like to evaluate this vector function at:

$x = [-2, 1, 0, 1, 2]$
$y = [0, 1, -3, 2, -1]$
$z = [10, -10, 0.1, -0.1, 1]$

You would instantiate the AutoDiff object as follows. The example below initiatives the seed to be 2 for all the partial derivatives. 

In [9]:
x = AutoDiff([-2, -1, 0, 1, 2], 2, n=3, k=1)
y = AutoDiff([0, 1, -3, 2, -1], 2, n=3, k=2)
z = AutoDiff([10, -10, 0.1, -0.1, 2], 1, n=3, k=3)

#### Building Up a Vector Function
Create functions as before.

In [11]:
f1 = 3*x**2 + x*y**2 - 2*z**3
f2 = y*ef.exp(x)
f3 = ef.cos(x*z) + ef.sin(y/z)

When vectorizing the functions, pass in the number of inputs and the number of vectors.

In [12]:
f = vectorize([f1, f2, f3], n_inputs = 3, n_vectors = 5)

print("Value:\n", f.val)
print("Derivative:\n", f.der)
print("Jacobian:\n", f.jacobian)

Value:
 [[-1.98800000e+03  2.00200000e+03 -2.00000000e-03  7.00200000e+00
  -2.00000000e+00]
 [ 0.00000000e+00  3.67879441e-01 -3.00000000e+00  5.43656366e+00
  -7.38905610e+00]
 [ 4.08082062e-01 -9.38904946e-01  1.98803162e+00  8.20589146e-02
  -1.13306916e+00]]
Derivative:
 [[[-2.40000000e+01  0.00000000e+00 -6.00000000e+02]
  [ 0.00000000e+00  2.70670566e-01  0.00000000e+00]
  [ 1.82589050e+01  2.00000000e-01 -1.82589050e+00]]

 [[-1.00000000e+01 -4.00000000e+00 -6.00000000e+02]
  [ 7.35758882e-01  7.35758882e-01  0.00000000e+00]
  [-1.08804222e+01 -1.99000833e-01 -5.53971153e-01]]

 [[ 1.80000000e+01  0.00000000e+00 -6.00000000e-02]
  [-6.00000000e+00  2.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  3.08502900e+00  4.62754350e+01]]

 [[ 2.00000000e+01  8.00000000e+00 -6.00000000e-02]
  [ 1.08731273e+01  5.43656366e+00  0.00000000e+00]
  [-1.99666833e-02 -8.16164124e+00 -8.15165789e+01]]

 [[ 2.60000000e+01 -8.00000000e+00 -2.40000000e+01]
  [-1.47781122e+01  1.47781122e+01  0.0

The derivative and jacobian will be a matrix in which the ith column is a matrix for the ith tuple of $(x_i, y_i, z_i)$.

<a id='dual_mode'></a>
## Calculating Higher Order Derivatives using Dual Mode

A dual number is in the form of:
$a + b\epsilon$

Where $\epsilon^2 = 0$

A nice property of dual numbers is that $b$ is the derivative of $a$.

For more information on dual numbers, please see the documentation in `/docs`.

<a id='dual_high_single'></a>
### Higher Order Derivatives for Single Variables

To calculate the $n^{th}$ derivative of functions with single variables, you can use the `Dual` object which holds a dual number.

#### Instantiating a Dual Object for Nth Derivatives
To instantiate a `Dual` object, you must pass in the value at which to evaluate the function. You may also pass in a seed of your choice, but must pass in a seed of 1 to evaluate the jacobian.  

In [39]:
x = Dual(3)
print(x)

3 + 1ε


x is a `Dual` object with a real value of 3 and a derivative of 1.

After instantiating the `Dual` object, decide to which order you would like to derive your function. Create a new higher order Dual Number by calling the class method `.makeHighestOrder(order)` on your original `Dual` object.

The code below builds a `Dual` object to evaluate up to the fourth derivative.

In [40]:
x4 = x.makeHighestOrder(4)

#### Building Up a Function
You can build up a function in the same manner as for the `AutoDiff` object. 

In [41]:
f = -5*x4**3

To get the values of $f(x)$ and the first to $n$ derivatives, call the `.buildCoefficients(order)` method on `f`. 

In [42]:
f.buildCoefficients(4)
print(f)

-135.0 + -135.0ε + -90.0ε^2 + -30.0ε^3 + -0.0ε^4


The first value in the vector is the value of the function. The second value in the vector is the first derivative. The third vaue is the second derivative, and so on. 

You can access each element of the vector in the following way:

In [43]:
print("Value: {}".format(f.coefficients[0]))
print("First derivative: {}".format(f.coefficients[1]))
print("Second derivative: {}".format(f.coefficients[2]))
print("Third derivative: {}".format(f.coefficients[3]))
print("Fourth derivative: {}".format(f.coefficients[4]))

Value: -135.0
First derivative: -135.0
Second derivative: -90.0
Third derivative: -30.0
Fourth derivative: -0.0


<a id='dual_high_vector'></a>
### Higher Order Derivatives for Single Variables of Vectors Inputs

You may pass in a `numpy` array of values at which to evaluate the function.

In [5]:
x = Dual(np.array([3, -0.8, -4, 0.1, 2]), 1)
x3 = x.makeHighestOrder(3)

<div class="alert alert-block alert-info">
<b>Warning:</b> Unlike in Forward Mode for <b>AutoDiff</b> objects, all vectors must be passed in to <b>Dual</b> objects using numpy arrays. 
</div>

In [10]:
f = 2*x3**5
f.buildCoefficients(3)
print(f)
print("Value: {}".format(f.coefficients[0]))
print("First derivative: {}".format(f.coefficients[1]))
print("Second derivative: {}".format(f.coefficients[2]))
print("Third derivative: {}".format(f.coefficients[3]))

[ 4.8600e+02 -6.5536e-01 -2.0480e+03  2.0000e-05  6.4000e+01] + [8.100e+02 4.096e+00 2.560e+03 1.000e-03 1.600e+02]ε + [ 1.080e+03 -2.048e+01 -2.560e+03  4.000e-02  3.200e+02]ε^2 + [1.08e+03 7.68e+01 1.92e+03 1.20e+00 4.80e+02]ε^3
Value: [ 4.8600e+02 -6.5536e-01 -2.0480e+03  2.0000e-05  6.4000e+01]
First derivative: [8.100e+02 4.096e+00 2.560e+03 1.000e-03 1.600e+02]
Second derivative: [ 1.080e+03 -2.048e+01 -2.560e+03  4.000e-02  3.200e+02]
Third derivative: [1.08e+03 7.68e+01 1.92e+03 1.20e+00 4.80e+02]


The $i^{th}$ column of each derivative vector corresponds to the $i^{th}$ value of passed in vector. Example: The third derivative of $f(x)$ at $x = -4$ is $1920$.

<a id='dual_high_vector_function'></a>
### Higher Order Derivatives for Vector Functions with Single Variables

As with the `AutoDiff` object, you can build derivatives of vector functions. 

In [2]:
x = Dual(2, 1)

x2 = x.makeHighestOrder(5)

f1 = 3*x2**5
f2 = 2*ef.sin(x2)
f3 = ef.exp(2*x2)

To pull the functions together in a vector function, call the function `vectorizeDual`. 

In [4]:
f = vectorizeDual([f1, f2, f3], 5)

The class attribute `.coefficients` holds a $nxm$ array where $n$ is the number of functions and $m$ holds the value and first derivative to the last derivative. (In this example, `.coefficients` is $3x6$ as it holds 1 value and 5 derivatives for each of the 3 functions. 

In [7]:
print("Value for f1: {}".format(f.coefficients[0, 0]))
print("First derivative for f1: {}".format(f.coefficients[0, 1]))
print("Second derivative for f1: {}".format(f.coefficients[0, 2]))
print("Third derivative for f1: {}".format(f.coefficients[0, 3]))
print("Fourth derivative for f1: {}".format(f.coefficients[0, 4]))
print("Fifth derivative for f1: {}".format(f.coefficients[0, 5]))

Value for f1: 96.0
First derivative for f1: 240.0
Second derivative for f1: 480.0
Third derivative for f1: 720.0
Fourth derivative for f1: 720.0
Fifth derivative for f1: 360.0
